In [1]:
import numpy as np

In [2]:
class Tensor:
    """
    Tensor
    """
    def __init__(self, numpy_array):
        if not isinstance(numpy_array, np.ndarray):
            raise ValueError("Must be initialized with a Numpy array")
        self.value = numpy_array
        self.previous_tensors = None
        self.operation = None  # type of operation done to create this tensor.
        self.grad = np.full(self.value.shape, 1)
        
    def backward(self):
        self._compute_grads(self)
    
    def _compute_grads(self, tensor):
        if tensor.previous_tensors is None:
            return
        else:
            t1, t2 = tensor.previous_tensors
            if tensor.operation == 'add':
                print('r add')
                t1.grad = tensor.grad
                t2.grad = tensor.grad
            elif tensor.operation == 'mul':
                t1.grad = tensor.grad * t2.value 
                t2.grad = tensor.grad * t1.value
            else:
                raise ValueError("Invalid operation")
            tensor.grad = None  # remove for saving memory
            self._compute_grads(t1)
            self._compute_grads(t2)
    
            
    def __add__(self, other):
        if isinstance(other, Tensor):
            # check shapes
            if not self.value.shape == other.value.shape:
                raise ValueError("Tensors must have the same shape. shape1: {}, shape2: {}".\
                                     format(str(self.value.shape), str(other.value.shape)))
            operation_result = self.value + other.value
            new_tensor = Tensor(operation_result)
            new_tensor.operation = 'add'
            new_tensor.previous_tensors = [self, other]
            print('add operation happened')
            return new_tensor
        else:
            raise ValueError("Operands must be of `Tensor` type.")
            
    
    def __mul__(self, other):
        if isinstance(other, Tensor):
            # check shapes
            if not self.value.shape == other.value.shape:
                raise ValueError("Tensors must have the same shape. shape1: {}, shape2: {}".\
                                     format(str(self.value.shape), str(other.value.shape)))
            operation_result = self.value * other.value
            new_tensor = Tensor(operation_result)
            new_tensor.operation = 'mul'
            new_tensor.previous_tensors = [self, other]
            print('mul operation happened')
            return new_tensor
        else:
            raise ValueError("Operands must be of `Tensor` type.")
    
    def __str__(self):
        return self.value.__repr__()

In [3]:
arr1 = np.random.rand(2, 3)
arr2 = np.random.rand(2, 3)

In [4]:
t1 = Tensor(arr1)
t2 = Tensor(arr2)
print('t1:', t1)
print('t2:', t2)

t1: array([[0.23020265, 0.02555293, 0.20129533],
       [0.62014705, 0.68127824, 0.4588507 ]])
t2: array([[0.21866268, 0.29939985, 0.1825974 ],
       [0.48037387, 0.72101232, 0.2129482 ]])


In [5]:
t3 = Tensor(np.full(arr1.shape, 10)) * t1 * Tensor(np.full(arr1.shape, 5)) * Tensor(np.full(arr1.shape, 5)) + t2 * Tensor(np.full(arr1.shape, 3))
t4 = (t3 + Tensor(np.full(arr1.shape, 2))) * Tensor(np.full(arr1.shape, 7))

mul operation happened
mul operation happened
mul operation happened
mul operation happened
add operation happened
add operation happened
mul operation happened


In [6]:
t4.backward()

r add
r add


In [7]:
t1.grad

array([[1750, 1750, 1750],
       [1750, 1750, 1750]])

In [8]:
t2.grad

array([[21, 21, 21],
       [21, 21, 21]])

In [9]:
t3.grad